In [1]:
import pandas as pd
from os.path import join

In [2]:
ROOT_DIR = '/home/jovyan/crop-forecasting'

train_df = pd.read_csv(join(ROOT_DIR, 'data', 'raw', 'train.csv'))
test_df = pd.read_csv(join(ROOT_DIR, 'data', 'raw', 'test.csv'))

In [3]:
crop_yields = train_df['Rice Yield (kg/ha)'].unique().tolist()

def rounded_yield(x, crop_yields):
    diffs = [abs(x - crop_yield) for crop_yield in crop_yields]
    return crop_yields[diffs.index(min(diffs))]

In [4]:
def get_other_yield(row):
    Season = 'Season(SA = Summer Autumn, WS = Winter Spring)'
    other_yield = train_df[(train_df['District'] == row['District']) & 
                           (train_df['Latitude'] == row['Latitude']) & 
                           (train_df['Longitude'] == row['Longitude']) & 
                           (train_df[Season] != row[Season])]['Rice Yield (kg/ha)'].tolist()

    if len(other_yield) > 0:
        res = other_yield[0]
    else:
        res = train_df[(train_df['District'] == row['District']) & 
                       (train_df[Season] != row[Season])]['Rice Yield (kg/ha)'].mean()
        
    return res

In [5]:
train_df['Other Rice Yield (kg/ha)'] = train_df.apply(get_other_yield, axis='columns')
train_df['Other Rice Yield (kg/ha)'] = train_df['Other Rice Yield (kg/ha)'].apply(lambda x: rounded_yield(x, crop_yields)).astype('int64')

train_df.to_csv(join(ROOT_DIR, 'data', 'interim', 'train_enriched.csv'))

In [6]:
test_df['Other Rice Yield (kg/ha)'] = test_df.apply(get_other_yield, axis='columns')
test_df['Other Rice Yield (kg/ha)'] = test_df['Other Rice Yield (kg/ha)'].apply(lambda x: rounded_yield(x, crop_yields)).astype('int64')

test_df.to_csv(join(ROOT_DIR, 'data', 'interim', 'test_enriched.csv'))